In [1]:
import os

import pandas as pd
import duckdb

In [2]:
for i in range(10):
    cwd = os.getcwd()
    r, c = os.path.split(cwd)
    if c == 'repo':
        print(f'new working dir: {cwd}')
        break
    os.chdir(r)

new working dir: c:\Users\aknof\Documents\GT\CSE_6250_BD4H\Project\repo


In [17]:
con = duckdb.connect()

CUI_MIN_OCC_POP = 1  #100
CUI_MAX_OCC_PAT = 10000
BILL_MIN_OCC_POP = 1  #1000

In [19]:
file_dir = os.path.join('data', 'nlp_jsl')
all_pat_df = None
for file in os.listdir(file_dir):
    patient = file.split('-')[1]
    patient_df = pd.read_csv(os.path.join(file_dir, file))
    patient_df['SUBJECT_ID'] = patient
    if all_pat_df is None:
        all_pat_df = patient_df
    else:
        all_pat_df = pd.concat([all_pat_df, patient_df])
    # break
all_pat_df.head()

,Unnamed: 0.1,Unnamed: 0,sentence,begin,end,chunks,entity,confidence,model,umls,SUBJECT_ID
0,0,0,36,2624,2653,part positional/lead placement,DRUG,0.702100,ner_ade_clinical,['C1140614'],4367
1,1,1,52,3791,3804,lead placement,DRUG,0.779950,ner_ade_clinical,['C1283151'],4367
2,2,2,56,4114,4127,lead placement,DRUG,0.836000,ner_ade_clinical,['C1283151'],4367
3,3,3,91,7368,7383,C/O posterior/lt,DRUG,0.627100,ner_ade_clinical,['C0856256'],4367
4,4,4,91,7403,7421,same w atc tylenol,DRUG,0.615425,ner_ade_clinical,['C3897300'],4367


In [26]:
all_pat_df['umls'] = all_pat_df['umls'].apply(lambda x: x.split("'")[1])
all_pat_df = all_pat_df[['umls', 'SUBJECT_ID']]
all_pat_df.head()

,umls,SUBJECT_ID
0,C1140614,4367
1,C1283151,4367
2,C1283151,4367
3,C0856256,4367
4,C3897300,4367


In [27]:
patient_df = con.execute(f"""
with high_vol as (
    select subject_id from all_pat_df
    group by subject_id
    having count(distinct umls) > {CUI_MAX_OCC_PAT}
)
select distinct umls, subject_id
from all_pat_df p
where not exists (select 1 from high_vol h where p.subject_id = h.subject_id)
""").df()
patient_df.head()

,umls,SUBJECT_ID
0,C0425573,83395
1,C3812244,83395
2,C4055623,83395
3,C0677491,83395
4,C1882136,83395


In [4]:
vocab_file = os.path.join('data', 'clinical_vocab', 'MRCONSO.RRF')
cols = ['CUI', 'LAT', 'TS', 'LUI', 'STT', 'SUI', 'ISPREF', 'AUI', 'SAUI',
    'SCUI', 'SDUI', 'SAB', 'TTY', 'CODE', 'STR', 'SRL', 'SUPPRESS', 'CVF']
umls = pd.read_csv(vocab_file, sep="|", 
header=None, names=cols, index_col=False)

umls.head()

C:\Users\aknof\AppData\Local\Temp\ipykernel_3896\2213249299.py:4: DtypeWarning: Columns (9,10,13) have mixed types. Specify dtype option on import or set low_memory=False.
  umls = pd.read_csv(vocab_file, sep="|",


,CUI,LAT,TS,LUI,STT,SUI,ISPREF,AUI,SAUI,SCUI,SDUI,SAB,TTY,CODE,STR,SRL,SUPPRESS,CVF
0,C0000005,ENG,P,L0000005,PF,S0007492,Y,A26634265,NaN,M0019694,D012711,MSH,PEP,D012711,(131)I-Macroaggregated Albumin,0,N,256.0
1,C0000005,ENG,S,L0270109,PF,S0007491,Y,A26634266,NaN,M0019694,D012711,MSH,ET,D012711,(131)I-MAA,0,N,256.0
2,C0000005,FRE,P,L6220710,PF,S7133957,Y,A13433185,NaN,M0019694,D012711,MSHFRE,PEP,D012711,Macroagrégats d'albumine marquée à l'iode 131,3,N,NaN
3,C0000005,FRE,S,L6215648,PF,S7133916,Y,A27488794,NaN,M0019694,D012711,MSHFRE,ET,D012711,MAA-I 131,3,N,NaN
4,C0000005,FRE,S,L6215656,PF,S7133956,Y,A27614225,NaN,M0019694,D012711,MSHFRE,ET,D012711,Macroagrégats d'albumine humaine marquée à l'i...,3,N,NaN


In [29]:
umls = con.execute(f"""
select CUI, row_number() over(order by CUI) as feature_idx
from umls u
where exists (select 1 from all_pat_df p where u.CUI = p.umls)
and LAT = 'ENG'
group by CUI
having count(*) >= {CUI_MIN_OCC_POP}
""").df()
umls.head()

,CUI,feature_idx
0,C0000734,1
1,C0002682,2
2,C0003835,3
3,C0005687,4
4,C0006318,5


In [30]:
umls.to_csv(os.path.join('data', 'consumed', 'feature_map.csv'))

In [31]:
pat_feat_long_df = con.execute("""
with patients as (
    select distinct subject_id from all_pat_df
)
select p.subject_id
,c.feature_idx
,count(pc.umls) feature_val
from umls c
cross join patients p
left join all_pat_df pc
on c.CUI = pc.umls
and p.subject_id = pc.subject_id
group by p.subject_id
,c.feature_idx
""").df()
pat_feat_long_df.head()

,SUBJECT_ID,feature_idx,feature_val
0,83395,1,4
1,83395,2,34
2,83395,3,26
3,83395,4,1
4,83395,5,4


In [34]:
pat_feat_wide = pat_feat_long_df.pivot(index='SUBJECT_ID', columns='feature_idx', values='feature_val')
pat_feat_wide = pat_feat_wide.reset_index()
pat_feat_wide.head(2)

feature_idx,SUBJECT_ID,1,2,3,4,5,6,7,8,9,...,1706,1707,1708,1709,1710,1711,1712,1713,1714,1715
0,4367,0,0,12,0,17,0,0,7,6,...,0,0,0,42,0,0,0,0,0,62
1,83395,4,34,26,1,4,4,12,5,5,...,28,8,1,58,5,5,4,1,1,8


In [ ]:
pat_feat_wide.to_csv(os.path.join('data', 'consumed', 'patient_feature.csv'), index=False)

In [26]:
# rel_file = os.path.join('data', 'clinical_vocab', '2022AB', 'META', 'MRREL.RRF')
# cols = ['CUI1', 'AUI1', 'STYPE1', 'REL', 'CUI2', 'AUI2', 'STYPE2', 'RELA', 'RUI',
#     'SRUI', 'SAB', 'SL', 'RG', 'DIR', 'SUPPRESS', 'CVF']
# umls_rel = pd.read_csv(rel_file, sep="|", 
# header=None, names=cols, index_col=False)

# umls_rel.head()

C:\Users\aknof\AppData\Local\Temp\ipykernel_20440\1973826217.py:4: DtypeWarning: Columns (7,9,13) have mixed types. Specify dtype option on import or set low_memory=False.
  umls_rel = pd.read_csv(rel_file, sep="|",


In [6]:
cpt = pd.read_csv(os.path.join('data', 'CPTEVENTS.csv'))
cpt.head()

C:\Users\aknof\AppData\Local\Temp\ipykernel_20440\3658660055.py:1: DtypeWarning: Columns (4,5,7,11) have mixed types. Specify dtype option on import or set low_memory=False.
  cpt = pd.read_csv(os.path.join('data', 'CPTEVENTS.csv'))


,ROW_ID,SUBJECT_ID,HADM_ID,COSTCENTER,CHARTDATE,CPT_CD,CPT_NUMBER,CPT_SUFFIX,TICKET_ID_SEQ,SECTIONHEADER,SUBSECTIONHEADER,DESCRIPTION
0,317,11743,129545,ICU,NaN,99232,99232.0,NaN,6.0,Evaluation and management,Hospital inpatient services,NaN
1,318,11743,129545,ICU,NaN,99232,99232.0,NaN,7.0,Evaluation and management,Hospital inpatient services,NaN
2,319,11743,129545,ICU,NaN,99232,99232.0,NaN,8.0,Evaluation and management,Hospital inpatient services,NaN
3,320,11743,129545,ICU,NaN,99232,99232.0,NaN,9.0,Evaluation and management,Hospital inpatient services,NaN
4,321,6185,183725,ICU,NaN,99223,99223.0,NaN,1.0,Evaluation and management,Hospital inpatient services,NaN


In [7]:
cpt = cpt[['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'CPT_CD', 'CPT_NUMBER', 'TICKET_ID_SEQ']]
cpt.head()

,ROW_ID,SUBJECT_ID,HADM_ID,CPT_CD,CPT_NUMBER,TICKET_ID_SEQ
0,317,11743,129545,99232,99232.0,6.0
1,318,11743,129545,99232,99232.0,7.0
2,319,11743,129545,99232,99232.0,8.0
3,320,11743,129545,99232,99232.0,9.0
4,321,6185,183725,99223,99223.0,1.0


In [8]:
dx = pd.read_csv(os.path.join('data', 'DIAGNOSES_ICD.csv'))
dx.head()

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,1297,109,172335,1.0,40301
1,1298,109,172335,2.0,486
2,1299,109,172335,3.0,58281
3,1300,109,172335,4.0,5855
4,1301,109,172335,5.0,4254


In [9]:
proc = pd.read_csv(os.path.join('data', 'PROCEDURES_ICD.csv'))
proc.head()

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,944,62641,154460,3,3404
1,945,2592,130856,1,9671
2,946,2592,130856,2,3893
3,947,55357,119355,1,9672
4,948,55357,119355,2,331


In [10]:
d_cpt = pd.read_csv(os.path.join('data', 'D_CPT.csv'))
d_cpt.head()

,ROW_ID,CATEGORY,SECTIONRANGE,SECTIONHEADER,SUBSECTIONRANGE,SUBSECTIONHEADER,CODESUFFIX,MINCODEINSUBSECTION,MAXCODEINSUBSECTION
0,1,1,99201-99499,Evaluation and management,99201-99216,Office/other outpatient services,NaN,99201,99216
1,2,1,99201-99499,Evaluation and management,99217-99220,Hospital observation services,NaN,99217,99220
2,3,1,99201-99499,Evaluation and management,99221-99239,Hospital inpatient services,NaN,99221,99239
3,4,1,99201-99499,Evaluation and management,99241-99255,Consultations,NaN,99241,99255
4,5,1,99201-99499,Evaluation and management,99261-99263,Follow-up inpatient consultations (deleted codes),NaN,99261,99263


In [11]:
cpt = con.execute("""
select c.SUBJECT_ID
,d.CATEGORY as cpt_cat
from cpt c
join d_cpt d
on c.cpt_number >= d.MINCODEINSUBSECTION
and c.cpt_number <= d.MAXCODEINSUBSECTION 
union
select c.SUBJECT_ID
,c.cpt_cd as cpt_cat
from cpt c
where c.cpt_number is null
""").df()
cpt.head()

,SUBJECT_ID,cpt_cat
0,59231,1
1,43187,1
2,92098,1
3,91383,1
4,69433,1


In [12]:
dx = con.execute("""
select distinct SUBJECT_ID, substring(ICD9_CODE, 1,
    case when left(ICD9_CODE, 1) in ('E', '0') then 4
    else 3 end --V, 1-9
) dx_group
from dx
where ICD9_CODE not like '0%'
and ICD9_CODE not like 'E%'
and ICD9_CODE not like 'V%'
""").df()
dx.head()

,SUBJECT_ID,dx_group
0,24210,424
1,24210,244
2,24211,864
3,24211,682
4,24214,276


In [14]:
pat_tgt_long = con.execute(f"""
with tgts as (
select SUBJECT_ID, cpt_cat as src_val, 'cpt' as src_type
from cpt
union all
select SUBJECT_ID, dx_group, 'dx' as src_type
from dx
union all
select distinct SUBJECT_ID, icd9_code, 'procedure' as src_type
from proc
)
select *, dense_rank() over (order by src_type, src_val) as target_idx
from tgts t
where exists (select 1
                from tgts cnt
                group by src_type, src_val
                having t.src_val = cnt.src_val
                and t.src_type = cnt.src_type
                and count(*) >= {BILL_MIN_OCC_POP})
""").df()
pat_tgt_long.head()

,SUBJECT_ID,src_val,src_type,target_idx
0,59231,1,cpt,1
1,43187,1,cpt,1
2,92098,1,cpt,1
3,91383,1,cpt,1
4,69433,1,cpt,1


In [15]:
target_map = con.execute("""
select src_val, src_type, target_idx 
from pat_tgt_long
group by src_val, src_type, target_idx 
--order by target_idx
""").df()
target_map.head()

,src_val,src_type,target_idx
0,357,dx,234
1,358,dx,235
2,359,dx,236
3,360,dx,237
4,361,dx,238


In [16]:
target_map.to_csv(os.path.join('data', 'consumed', 'target_map.csv'))

In [17]:
target_map.count()

src_val       2803
src_type      2803
target_idx    2803
dtype: int64

In [18]:
pat_tgt_full = con.execute("""
with pat as (select distinct subject_id from pat_tgt_long)
select p.subject_id
,t.target_idx
,case when pt.target_idx is not null then 1 else 0 end as target_val
from target_map t
cross join pat p
left join pat_tgt_long pt
on t.target_idx = pt.target_idx
and p.subject_id = pt.subject_id
""").df()
pat_tgt_full.head()

,SUBJECT_ID,target_idx,target_val
0,60907,282,1
1,60907,284,1
2,60907,321,1
3,60907,324,1
4,60907,353,1


In [19]:
pat_tgt_wide = pat_tgt_full.pivot(index='SUBJECT_ID', columns='target_idx', values='target_val')
pat_tgt_wide = pat_tgt_wide.reset_index()
pat_tgt_wide.head()

target_idx,index,SUBJECT_ID,1,2,3,4,5,6,7,8,...,2794,2795,2796,2797,2798,2799,2800,2801,2802,2803
0,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,3,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,4,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,6,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
pat_tgt_wide = pat_tgt_wide.iloc[:, 1:]
print(pat_tgt_wide.columns[:5], pat_tgt_full.columns)
pat_tgt_wide.head()

Index(['SUBJECT_ID', 1, 2, 3, 4], dtype='object', name='target_idx') Index(['SUBJECT_ID', 'target_idx', 'target_val'], dtype='object')


target_idx,SUBJECT_ID,1,2,3,4,5,6,7,8,9,...,2794,2795,2796,2797,2798,2799,2800,2801,2802,2803
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,6,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
pat_tgt_wide.to_csv(os.path.join('data', 'consumed', 'patient_target.csv'), index=False)

In [105]:
# pat_tgt = pd.DataFrame(pat_tgt_wide['SUBJECT_ID'])
# pat_tgt['targets'] = (pat_tgt_wide.iloc[:,2:].astype(str)
#                 .agg(lambda x: ' '.join(i for i in x), axis=1)
#                 .str.split())
# pat_tgt.head()                

,SUBJECT_ID,targets
0,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,3,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,4,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,5,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,6,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [102]:
# pat_tgt.to_csv(os.path.join('data', 'consumed', 'patient_target.csv'), index=False)